In [1]:
import buffer_averaging
from appgeopy import *
from my_packages import *

In [2]:
mainfile_fpath = r"D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\calibrated\calibrated_S1A_tsERA5_avgvelocity.pkl"
main_gdf = pd.read_pickle(mainfile_fpath, compression="zip")
main_gdf = geospatial.convert_to_geodata(
    df=main_gdf,
    xcoord_col="POINT_X",
    ycoord_col="POINT_Y",
    crs_epsg="EPSG:3826",
)
print(type(main_gdf))
main_gdf.head(3)

<class 'geopandas.geodataframe.GeoDataFrame'>


,OBJECTID,CODE,HEIGHT,H_STDEV,VEL,V_STDEV,COHERENCE,EFF_AREA,POINT_X,POINT_Y,PointKey,velocity_mm_per_day,velocity_mm_per_year,geometry
0,1,00000001,20.549999,0.0,23.73,7.10,0.972,1,174062.9530,2.636744e+06,X174062Y2636744,-0.025057,-9.152233,POINT (174062.953 2636744.230)
1,23,00000017,32.950001,0.0,15.85,4.24,0.987,1,184876.0580,2.634928e+06,X184876Y2634927,-0.025778,-9.415311,POINT (184876.058 2634927.562)
2,26,0000001a,35.020001,0.0,-46.50,8.12,0.985,1,185416.2562,2.634911e+06,X185416Y2634911,-0.104221,-38.066608,POINT (185416.256 2634911.408)


In [3]:
slavefile_fpath = r"D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\calibrated\calibrated_TSX_tsERA5_avgvelocity.pkl"
slave_gdf = pd.read_pickle(slavefile_fpath, compression="zip")
slave_gdf = geospatial.convert_to_geodata(
    df=slave_gdf,
    xcoord_col="POINT_X",
    ycoord_col="POINT_Y",
    crs_epsg="EPSG:3826",
)
print(type(slave_gdf))
slave_gdf.head(3)

<class 'geopandas.geodataframe.GeoDataFrame'>


,OBJECTID,CODE,HEIGHT,H_STDEV,VEL,V_STDEV,COHERENCE,EFF_AREA,POINT_X,POINT_Y,PointKey,velocity_mm_per_day,velocity_mm_per_year,geometry
0,1,00000001,28.020000,0.0,2.24,7.28,0.712,1,178147.1514,2.630709e+06,X178147Y2630708,-0.206489,-75.420055,POINT (178147.151 2630708.793)
1,2,00000002,27.969999,0.0,8.71,7.63,0.714,1,178159.5813,2.630706e+06,X178159Y2630706,-0.151533,-55.347293,POINT (178159.581 2630706.407)
2,3,00000003,27.570000,0.0,11.59,13.49,0.689,1,178172.0111,2.630704e+06,X178172Y2630704,0.224200,81.889164,POINT (178172.011 2630704.022)


In [5]:
# Split the main GeoDataFrame into smaller chunks
chunk_size = 10_000
chunks = buffer_averaging.split_gdf(main_gdf, chunk_size)
len(chunks)

18

In [ ]:
# Use multiprocessing to process each chunk
results = []
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    futures = [executor.submit(process_chunk, chunk, slave_gdf, slave_measurement_col, buffer_radius) for chunk in chunks]
    for future in as_completed(futures):
        results.append(future.result())

In [ ]:
# Combine all results into a single GeoDataFrame
final_gdf = gpd.GeoDataFrame(pd.concat(results, ignore_index=True), crs=main_gdf.crs)

# Save the result to a new shapefile
final_gdf.to_file(output_shapefile)

print(f"Output saved to {output_shapefile}")